# HCMUS-UNIV-WebScience-VisualizeData

## PROJECT 2: MỐI QUAN HỆ CỦA DỮ LIỆU


### I. Chuẩn bị môi trường:

In [262]:
import sqlite3
import pandas as pd
import json

In [263]:
def fetch_value(sqlCommand):
    conn = sqlite3.connect("data.db",check_same_thread=False)
    ret = pd.read_sql_query(sqlCommand, conn)
    conn.close()
    return ret

# Query dữ liệu
def query_sql(sql:str):
    listData = fetch_value(sql)
    return listData

# Lấy bài viết theo website.
def get_bai_viet(website:str):
    return query_sql(f"select * from baiviet where website='{website}'")

class Utils:
    def convertHMtoFloat(HM):
        h,m = HM.split(":")
        return int(h)+(float(m)/60)

### I. Tiền xử lý dữ liệu

 #### Yêu cầu: đưa các field về dạng chuẩn chung:

1. active_time: thời gian hoạt động, đưa về dạng sau:

```
    {
        <tên ngày trong tuần>:[ <thời gian mở cửa>,<thời gian đóng cửa> ]
    }

    *lowercase
    *thời gian đưa về float: 8h30 => 8.5
```
    
- ví dụ:

```
    {
        "thứ 2": [7,21.5],
        ...,
        "chủ nhật": [8,22.3]
    }
```

2. other_service: các dịch vụ khác, đưa về dạng sau:

```
    [
        <tên dịch vụ>,
        <tên dịch vụ>,
        ...,
    ]
    
    *lowercase
```
    
- ví dụ:

```
    [
        "có wifi"
        "giao hàng",
        ...,
    ]
```

#### I.1 Foody

In [264]:
def preprocessing_Foody():
    # Code here
    data = get_bai_viet("Foody")
    active_time = data["active_time"]
    other_service = data["other_service"]
    format_time = []
    format_other = []
    for time in active_time:
        #time = time.open + ',' + time.close
        time = json.loads(time)
        time_format = []
        time_format.append(time['open'])
        time_format.append(time['close'])
        x = {
            "thứ 2":time_format,
            "thứ 3":time_format,
            "thứ 4":time_format,
            "thứ 5":time_format,
            "thứ 6":time_format,
            "thứ 7":time_format,
            "chủ nhật":time_format,
            }
        time = json.dumps(x)
        dict_time = json.loads(time)
        format_time.append(dict_time)
    data['active_time'] = format_time
    for other in other_service:
        other = other.lower()
        other = json.loads(other)
        format_other.append(other)
    data['other_service'] = format_other
    return data

#### I.2 ShopeeFood

In [265]:
def preprocessing_ShopeeFood():
    def maping(name):
        if name=="monday":
            return "thứ 2"
        if name=="tuesday":
            return "thứ 3"
        if name=="wednesday":
            return "thứ 4"
        if name=="thursday":
            return "thứ 5"
        if name=="friday":
            return "thứ 6"
        if name=="saturday":
            return "thứ 7"
        if name=="sunday":
            return "chủ nhật"

    def convert_active_time(HM):
        pre_format = {}
        at_js = json.loads(HM)
        atl = list(at_js)
        for key in atl:
            open,close = at_js[key].split("-")
            pre_format[maping(key)] = [Utils.convertHMtoFloat(open),Utils.convertHMtoFloat(close)]
        return pre_format

    data = get_bai_viet("ShopeeFood")
    data["active_time"] = data["active_time"].apply(convert_active_time)

    return data

#### I.3 GoogleReview

In [266]:
def preprocessing_GoogleReview():
    # Code here
    return pd.DataFrame()

#### I.4 Tổng hợp lại dữ liệu tiền xử lý

In [267]:
DATA = pd.concat([preprocessing_Foody(),preprocessing_ShopeeFood(),preprocessing_GoogleReview()])
#preprocessing_Foody()
#Sumary statistic
# DATA.describe()

### 2. Thống kê dữ liệu

// Mô tả ...

In [270]:
# Code here
DATA.sort_values(by="rate", ascending=False).head()
#Những quán ăn Quận 1 mà được lượt rate cao tới thấp
DATA[DATA["district"].apply(lambda state: state[0] == "1")].sort_values(by="rate", ascending=False).head()

#Những quán có rate cao là số favorite cũng cao
columns_to_show = ["rate", "favorite", "district"]
DATA.sort_values(by="rate", ascending=False).groupby(["full_name"])[columns_to_show].describe(percentiles=[])

#Những quán có giá đắt
columns_show = ["full_name","price_from","price_to",]
DATA.sort_values(by="price_from",ascending=False).groupby(["full_name"])[columns_show].describe(percentiles=[])

price_from                          \
                                                count      mean std       min   
full_name                                                                       
 Lẩu Bò Q7 - La Thành                             1.0       0.0 NaN       0.0   
 MIFASHOP - Mỹ Phẩm, Nước Hoa Chính Hãng          1.0  100000.0 NaN  100000.0   
 Sly1994                                          1.0  100000.0 NaN  100000.0   
 Sơn Mập Quán - Heo Mẹt - Đường Số 11             1.0   50000.0 NaN   50000.0   
1 Food Drink                                      1.0   50000.0 NaN   50000.0   
...                                               ...       ...  ..       ...   
Ốc Đêm Nha Trang                                  1.0   40000.0 NaN   40000.0   
Ốc Đại Dương - Nguyễn Văn Luông                   1.0   15000.0 NaN   15000.0   
Ốc Đẹp                                            1.0   20000.0 NaN   20000.0   
Ớt Xanh 1 - Cháo Ếch Singapore                    1.0   20000.0 NaN   20000.0   
Ụt Ụt Quán - Barbecue & Beer - Võ Văn Kiệt        1.0  100000.0 NaN  100000.0   

                                                               price_to  \
                                                 50%       max    count   
full_name                                                                 
 Lẩu Bò Q7 - La Thành                            0.0       0.0      1.0   
 MIFASHOP - Mỹ Phẩm, Nước Hoa Chính Hãng    100000.0  100000.0      1.0   
 Sly1994                                    100000.0  100000.0      1.0   
 Sơn Mập Quán - Heo Mẹt - Đường Số 11        50000.0   50000.0      1.0   
1 Food Drink                                 50000.0   50000.0      1.0   
...                                              ...       ...      ...   
Ốc Đêm Nha Trang                             40000.0   40000.0      1.0   
Ốc Đại Dương - Nguyễn Văn Luông              15000.0   15000.0      1.0   
Ốc Đẹp                                       20000.0   20000.0      1.0   
Ớt Xanh 1 - Cháo Ếch Singapore               20000.0   20000.0      1.0   
Ụt Ụt Quán - Barbecue & Beer - Võ Văn Kiệt  100000.0  100000.0      1.0   

                                                                      \
                                                 mean std        min   
full_name                                                              
 Lẩu Bò Q7 - La Thành                             0.0 NaN        0.0   
 MIFASHOP - Mỹ Phẩm, Nước Hoa Chính Hãng    5000000.0 NaN  5000000.0   
 Sly1994                                     200000.0 NaN   200000.0   
 Sơn Mập Quán - Heo Mẹt - Đường Số 11         30000.0 NaN    30000.0   
1 Food Drink                                 150000.0 NaN   150000.0   
...                                               ...  ..        ...   
Ốc Đêm Nha Trang                             100000.0 NaN   100000.0   
Ốc Đại Dương - Nguyễn Văn Luông              200000.0 NaN   200000.0   
Ốc Đẹp                                        30000.0 NaN    30000.0   
Ớt Xanh 1 - Cháo Ếch Singapore               150000.0 NaN   150000.0   
Ụt Ụt Quán - Barbecue & Beer - Võ Văn Kiệt   300000.0 NaN   300000.0   

                                                                  
                                                  50%        max  
full_name                                                         
 Lẩu Bò Q7 - La Thành                             0.0        0.0  
 MIFASHOP - Mỹ Phẩm, Nước Hoa Chính Hãng    5000000.0  5000000.0  
 Sly1994                                     200000.0   200000.0  
 Sơn Mập Quán - Heo Mẹt - Đường Số 11         30000.0    30000.0  
1 Food Drink                                 150000.0   150000.0  
...                                               ...        ...  
Ốc Đêm Nha Trang                             100000.0   100000.0  
Ốc Đại Dương - Nguyễn Văn Luông              200000.0   200000.0  
Ốc Đẹp                                        30000.0    30000.0  
Ớt Xanh 1 - Cháo Ếch Singapore              

###  Phần 3,4,5